In [1]:

import numpy as np

import time
import hopsworks
import os
from dotenv import load_dotenv
import joblib
from datetime import datetime, timedelta, date
import pandas as pd

import hopsworks
import matplotlib.pyplot as plt

from utils import *
import utils
from functools import reduce



In [2]:
load_dotenv()
#os.environ["HOPSWORKS_API_KEY"] = os.getenv("HOPSWORKS_API_KEY")


project = hopsworks.login()

2025-01-04 22:08:28,672 INFO: Initializing external client
2025-01-04 22:08:28,673 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-04 22:08:30,383 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1160346


In [3]:

fs = project.get_feature_store() 
#secrets = secrets_api(project.name)

# This line will fail if you have not registered the CRYPTO_API_KEY as a secret in Hopsworks
#CRYPTO_API_KEY = secrets.get_secret("CRYPTO_API_KEY").value

#CRYPTO_API_KEY = os.getenv("CRYPTO_API_KEY")



In [4]:
#Step 1: get the  forcasted data
solana_pred_feature_group = fs.get_feature_group(
    name="solana_predictions",  
    version=1
)

#Step 2: get the actual data
solana_feature_group = fs.get_feature_group(
    name="solana",  
    version=1
)




In [5]:
# Step 3: Read the data from the feature group
solana_pred_df = solana_pred_feature_group.select(["date","open_solana"]).read()

# Step 4: Read the data from the feature group
solana_actual_df = solana_feature_group.select(["timestamp","open"]).read()

2025-01-04 22:08:34,574 ERROR: No commits found for featuregroup: /apps/hive/warehouse/id2223_airquality_lab1_featurestore.db/solana_predictions_1. Detail: Python exception: FlyingDuckException. gRPC client debug context: UNKNOWN:Error received from peer ipv4:51.79.26.27:5005 {created_time:"2025-01-04T22:08:34.567625+02:00", grpc_status:2, grpc_message:"No commits found for featuregroup: /apps/hive/warehouse/id2223_airquality_lab1_featurestore.db/solana_predictions_1. Detail: Python exception: FlyingDuckException"}. Client context: IOError: Server never sent a data message. Detail: Internal
Traceback (most recent call last):
  File "/Users/michailroussos/Desktop/MSc-courses/2024-2025_P2/Saclable Machine Learning and Deep Learning (ID2223)/Assignments/final project/Scalable Final Project/ID2223-final-project/venv/lib/python3.9/site-packages/hsfs/core/arrow_flight_client.py", line 364, in afs_error_handler_wrapper
    return func(instance, *args, **kw)
  File "/Users/michailroussos/Deskt

FeatureStoreException: No commits found for featuregroup: /apps/hive/warehouse/id2223_airquality_lab1_featurestore.db/solana_predictions_1. Detail: Python exception: FlyingDuckException. gRPC client debug context: UNKNOWN:Error received from peer ipv4:51.79.26.27:5005 {created_time:"2025-01-04T22:08:34.567625+02:00", grpc_status:2, grpc_message:"No commits found for featuregroup: /apps/hive/warehouse/id2223_airquality_lab1_featurestore.db/solana_predictions_1. Detail: Python exception: FlyingDuckException"}. Client context: IOError: Server never sent a data message. Detail: Internal

In [ ]:
solana_pred_df['date'] = pd.to_datetime(solana_pred_df['date'], errors='coerce')

solana_pred_df['date'] = solana_pred_df['date'].dt.normalize()
solana_pred_df['date'] = solana_pred_df['date'].dt.tz_localize(None)
solana_pred_df.rename(columns={'open_solana': 'solana_open_pred'}, inplace=True)
solana_pred_df = solana_pred_df.sort_values(by='date')

solana_pred_df



In [ ]:

solana_actual_df['date'] = pd.to_datetime(solana_actual_df['timestamp'], unit='s')
solana_actual_df['date'] = pd.to_datetime(solana_actual_df['date'], errors='coerce')

solana_actual_df['date'] = solana_actual_df['date'].dt.normalize()
solana_actual_df['date'] = solana_actual_df['date'].dt.tz_localize(None)
solana_actual_df.drop(columns=["timestamp"], inplace=True)
solana_actual_df.rename(columns={'open': 'solana_open_actual'}, inplace=True)
solana_actual_df = solana_actual_df.sort_values(by='date')


solana_actual_df

In [ ]:
file_path='img/solana_predictions_plot.png'

plot_solana_predictions(solana_pred_df, file_path) 

In [ ]:

solana_actual_df['date'] = pd.to_datetime(solana_actual_df['date'], errors='coerce')

solana_actual_df['date'] = solana_actual_df['date'].dt.normalize()
solana_actual_df['date'] = solana_actual_df['date'].dt.tz_localize(None)

solana_pred_df['date'] = pd.to_datetime(solana_pred_df['date'], errors='coerce')

solana_pred_df['date'] = solana_pred_df['date'].dt.normalize()
solana_pred_df['date'] = solana_pred_df['date'].dt.tz_localize(None)

In [ ]:
# Merge the two df on date
pred_actual_join = solana_pred_df.merge(solana_actual_df, left_on='date', right_on='date')

# Show the results
pred_actual_join

In [ ]:
file_path='img/solana_actual_vs_predictions_plot.png'
confidence_upper = [p + 1 for p in pred_actual_join['solana_open_pred']]
confidence_lower = [p - 1 for p in pred_actual_join['solana_open_pred']]
plot_solana_actual_vs_predictions(pred_actual_join, confidence_lower, confidence_upper, file_path)


In [ ]:
bitcoin_feature_group = fs.get_feature_group(
    name="bitcoin",  
    version=1
)

# Read the data from the feature group
btc_df = bitcoin_feature_group.select(["timestamp","open"]).read()

In [ ]:
btc_df['date'] = pd.to_datetime(btc_df['timestamp'], unit='s')
btc_df['date'] = pd.to_datetime(btc_df['date'], errors='coerce')

btc_df['date'] = btc_df['date'].dt.normalize()
btc_df['date'] = btc_df['date'].dt.tz_localize(None)
btc_df.drop(columns=["timestamp"], inplace=True)
btc_df.rename(columns={'open': 'btc_open'}, inplace=True)
#btc_df = btc_df.sort_values(by='date')


btc_df

In [ ]:
fng_feature_group = fs.get_feature_group(
    name='f_n_g_index', 
    version=1)
fng_df = fng_feature_group.select(["timestamp","fng_value"]).read()

In [ ]:
fng_df['date'] = pd.to_datetime(fng_df['timestamp'], unit='s')
fng_df['date'] = pd.to_datetime(fng_df['date'], errors='coerce')

fng_df['date'] = fng_df['date'].dt.normalize()
fng_df['date'] = fng_df['date'].dt.tz_localize(None)
fng_df.drop(columns=["timestamp"], inplace=True)
#fng_df.rename(columns={'value': 'fng_value'}, inplace=True)
#fng_df = fng_df.sort_values(by='date')


fng_df

In [ ]:

# List of DataFrames to merge
dfs = [btc_df, solana_actual_df, fng_df]

# Merge all DataFrames on 'date'
sln_btc_fng_join = reduce(lambda left, right: left.merge(right, on='date', how='inner'), dfs)
sln_btc_fng_join = sln_btc_fng_join.sort_values(by='date')

sln_btc_fng_join

In [ ]:
file_path='img/sln_btc_fng_heatmap.png'

plot_correlation_heatmap(sln_btc_fng_join,file_path)

In [ ]:
# List of DataFrames to merge
dfs = [btc_df, solana_actual_df,solana_pred_df, fng_df]

# Merge all DataFrames on 'date'
trend_join = reduce(lambda left, right: left.merge(right, on='date', how='inner'), dfs)
trend_join = trend_join.sort_values(by='date')

trend_join.columns

In [ ]:
file_path='img/trend_analisis.png'

plot_trend_analysis(trend_join,file_path)